In [ ]:
#제품 이미지를 생성하고 해당 제품을 홍보하기 위한 매력적인 문구를 만들어 주는 봇을 만들기
!pip install openai==1.10.0 gradio==4.0.2 langchain==0.1.4 langchain-openai==0.0.5

In [ ]:
import gradio as gr
import openai
import os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

# OpenAI API 키 설정

openai.api_key = "" # 이전에 발급받았던 OpenAI API 키를 넣어주세요

os.environ["OPENAI_API_KEY"] = openai.api_key

client = openai.OpenAI()

    # 입력받은 텍스트를 영어로 번역해 주는 함수
def translate_text(text):
    chat = ChatOpenAI(temperature=0)

    template="You are a helpful assistant that translates {input_language} to {output_language}."
    system_message_prompt = SystemMessagePromptTemplate.from_template(template)

    human_template="{text}"
    human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

    chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

    response = chat(
    chat_prompt.format_prompt(
        input_language="korean", output_language="english", text=text
    ).to_messages()
)

    return response.content

def generate_contents(prompt, size):
    translated_prompt = translate_text(prompt)

    img_prompt = [
        {"role": "system",
                 "content": "당신은 비즈니스 고객을 위한 목업 이미지를 만들어 주는 사람입니다. \
        사용자로부터 제품에 대한 설명을 듣고 제품에 대한 이미지를 만듭니다. \
        제품 이미지에 텍스트는 없어야 합니다."},
        {"role": "user", "content": f"(realistic){prompt}"}
    ]

    # 이미지 생성
    img_response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=img_prompt
    )

    img_prompt_result = img_response.choices[0].message.content

    dalle_response = client.images.generate(
        model="dall-e-3",
        prompt = img_prompt_result,
        size = size,
        n = 1
    )

    image_url = dalle_response.data[0].url

    # 홍보 문구 생성

    ad_prompt = [
        {"role": "system", "content": "당신은 제품 홍보 문구를 작성하는 카피라이터 입니다. \
        전달받은 내용을 분석해 100자 이내의 한글로 오로지 제품만을 매력있게 설명하세요."},
        {"role": "user", "content": f"{img_prompt_result}"}
    ]
    ad_response =  client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=ad_prompt
    )

    ad_text = ad_response.choices[0].message.content


    return ad_text, image_url

# 이미지 크기 선택을 위한 드롭다운 메뉴
image_sizes = gr.Dropdown(choices=["1024x1024", "1024x1792", "1792x1024"], label="이미지 크기")
iface = gr.Interface(
    fn=generate_contents,
    inputs=["text", image_sizes],
    outputs=["text","image"],
    live=False,  # live 모드 비활성화
    title="나만의 제품 홍보 봇!",
    description="사용자 프롬프트를 기반으로 ChatGPT와 DALL-E를 사용하여 이미지와 홍보 문구를 생성합니다. submit 버튼을 눌러 결과를 확인하세요."
)
iface.launch(debug = True)